In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
data_dir = Path("../data/processed")

### Load & standardize

In [4]:
sp500 = (
    pd.read_csv(data_dir / "sp500_merged.csv")
      .rename(columns={'Adj_Close_Combined':'sp500'})
      .assign(Date=lambda df: pd.to_datetime(df['Date']))
      .set_index('Date')[['sp500']]
)

In [ ]:
copper = (
    pd.read_csv(data_dir / "copper_monthly_cleaned.csv")
      .rename(columns={'copper_price':'copper'})
      .assign(date=lambda df: pd.to_datetime(df['date'], format='%m/%d/%Y'))
      .set_index('date')[['copper']]
)

In [6]:
def load_fred(file, col_new):
    df = pd.read_csv(data_dir / file, index_col=0, parse_dates=True)
    return df.rename(columns={df.columns[0]: col_new})

In [7]:
dgs10  = load_fred("dgs10.csv",   "dgs10")
tb3ms  = load_fred("tb3ms.csv",   "tb3ms")
oil    = load_fred("wtisplc.csv", "oil")
vix    = load_fred("vixcls.csv",  "vix")

In [28]:
# ---------- 0. helper --------------------------------------------------------
def to_month_end(df):
    out = df.copy()
    out.index = out.index.to_period('M').to_timestamp('M')
    return out

In [29]:
# ---------- 1. month-end everything first -----------------------------------
sp500  = to_month_end(sp500)
copper = to_month_end(copper)
dgs10  = to_month_end(dgs10)
tb3ms  = to_month_end(tb3ms)
oil    = to_month_end(oil)
vix    = to_month_end(vix)

In [36]:
# realised vol (pre-1990 proxy) ---------------------------------------------
sp500_daily['ret'] = sp500_daily['adj_close'].pct_change()
sp500_daily['realized_vol'] = sp500_daily['ret'].rolling(21).std() * np.sqrt(252)
realized_vol_m = (
    sp500_daily['realized_vol']
      .resample('ME').last()
      .to_frame('market_vol')
      .pipe(to_month_end)
)

In [37]:
# overwrite with VIX from Jan-1990 onward
vix_m = vix.resample('ME').last().rename(columns={'vix': 'market_vol'})
market_vol = realized_vol_m.combine_first(vix_m)   # realised <1990, VIX ≥1990


In [38]:
# ---------- 2. derived series -----------------------------------------------
yield_curve = (dgs10['dgs10'] - tb3ms['tb3ms']).to_frame('yield_curve')

In [39]:
# ---------- 3. assemble raw panel -------------------------------------------
raw_macro = (
    sp500.join([
        copper,
        oil,
        tb3ms.rename(columns={'tb3ms': 'policy_rate'}),
        dgs10,                        # keep raw 10-y yield
        yield_curve,
        market_vol
    ], how='outer')                  # <-- keep the full history
).sort_index()

In [40]:
# ---------- 4. stock-bond 36-m corr -----------------------------------------
sp500_ret = raw_macro['sp500'].pct_change()
dgs10_chg = raw_macro['dgs10'].diff()
raw_macro['stock_bond_corr'] = (
    sp500_ret.rolling(36, min_periods=36).corr(dgs10_chg)
)

In [41]:
# ---------- 5. save ----------------------------------------------------------
raw_macro.to_csv(data_dir / "raw_macro_monthly.csv")
print("▶ raw_macro saved:", raw_macro.shape, "rows/cols")

▶ raw_macro saved: (1169, 8) rows/cols
